In [1]:
%load_ext autoreload
%autoreload 1 
%aimport ncaa_pbp

from utility import get_Teams, base_url
from ncaa_pbp import Team
from tqdm import tqdm
import pandas as pd
import re

In [2]:
teams_dict = get_Teams(2024)
teams_dict = {name:tattrs[0].split('/')[-1] for name, tattrs in teams_dict.items()}


In [3]:
teams = []
cnt = 0
for team, teamid in tqdm(teams_dict.items()):
    if cnt < 218:
        teams.append(Team(teamid, loadPlayerStats=True,source='file'))
    else:
        teams.append(Team(teamid, loadPlayerStats=True,source='web'))
    cnt+=1

  0%|          | 0/360 [00:00<?, ?it/s]

100%|██████████| 360/360 [12:21<00:00,  2.06s/it]


In [20]:
playerStats = pd.DataFrame()
for team in teams:
    df = team.playerStats.iloc[:-3].copy()
    df['Team'] = team.teamId
    playerStats = pd.concat([playerStats, df])

In [28]:
def minToSec(time: str):
    minSec = time.split(':')
    return int(minSec[0]) * 60 + int(minSec[1])
playerStats['Seconds Played'] = playerStats.MP.apply(minToSec)
playerStats['Team Name'] = playerStats.Team.map({teamid:team for teamid, team in zip(teams_dict.values(), teams_dict.keys())})

In [40]:
playerStats['Sec/PF'].round(1)

0      541.4
1      583.9
2      562.8
3      801.9
4      356.7
       ...  
8      833.5
9      424.2
10     422.0
11    1154.9
12     832.2
Name: Sec/PF, Length: 4684, dtype: float64

In [57]:
playerStats['Sec/PF'] = playerStats['Seconds Played'] / playerStats.PF
playerStats['Sec/PF'] = playerStats['Sec/PF'].round(1)
playerStats = playerStats[['Player','Yr','Pos','Ht','GP','GS','MP','PF','Seconds Played','Team Name','Sec/PF']]
playerStats['GP'] = playerStats.GP.astype(int)
playerStats['MP/Game'] = playerStats['Seconds Played'] / 60 / playerStats['GP']

In [77]:
fouls = playerStats.sort_values('Sec/PF', ascending=True).query('GP>20').query('`MP/Game`>25').reset_index(drop=True)
print(fouls.query('Player=="Kennedy Calhoun"').index)
fouls.head(25)

Index([20], dtype='int64')


,Player,Yr,Pos,Ht,GP,GS,MP,PF,Seconds Played,Team Name,Sec/PF,MP/Game
0,Saniya Jackson,Fr,F,6-0,30,26,780:21,110.0,46821,Valparaiso,425.6,26.011667
1,Anajah Brown,Jr,F,6-1,23,22,577:49,81.0,34669,Siena,428.0,25.122464
2,Eleyana Tafisi,Jr,G,5-6,29,26,737:21,103.0,44241,Utah Valley,429.5,25.425862
3,Emma Broermann,Sr,F,6-3,30,30,791:56,107.0,47516,North Florida,444.1,26.397778
4,Jaylin Foster,Jr,F,6-0,32,32,802:01,108.0,48121,Texas St.,445.6,25.063021
5,Anna Foley,Fr,C,6-3,31,31,859:59,115.0,51599,Quinnipiac,448.7,27.741398
6,Taiyanna Jackson,Sr,C,6-6,32,32,860:34,112.0,51634,Kansas,461.0,26.892708
7,Sanaa' St. Andre,Sr,G,5-8,28,27,796:59,103.0,47819,Tennessee St.,464.3,28.463690
8,Mya Glanton,Sr,F,6-0,32,32,878:48,113.0,52728,Indiana St.,466.6,27.462500
9,Evelien Lutje Schipholt,Sr,F,6-2,33,28,833:56,107.0,50036,South Fla.,467.6,25.270707


In [78]:
print((1049*60+23)/100/60)
print((1156*60+35)/120/60)
print((1168*60+22)/140/60)


10.493833333333333
9.638194444444444
8.345476190476191


In [79]:
.64*60

38.4

In [80]:
.35*60

21.0

Technical Fouls

In [ ]:
playerStats.loc[~playerStats['Tech Fouls'].isna()].sort_values('Tech Fouls', ascending=False).head()

Heights

In [82]:
heights = playerStats.loc[:].copy()

In [88]:
def heightToInches(height):
    footInches = height.split('-')
    return int(footInches[0]) * 12 + int(footInches[1])
heights = heights.loc[heights.Ht != '-']
heights['Inches'] = heights.Ht.apply(heightToInches)


In [102]:
for row in heights.groupby('Team Name').Inches.mean().sort_values(ascending=True).items():
    print(row)

('Detroit Mercy', 67.86666666666666)
('Central Conn. St.', 68.2)
('Austin Peay', 68.5)
('Saint Francis', 68.63636363636364)
('Southeastern La.', 68.6923076923077)
('Alcorn', 68.71428571428571)
('Marshall', 68.8)
('SIUE', 68.81818181818181)
('Niagara', 68.92307692307692)
('UMass Lowell', 69.0)
('Hofstra', 69.0)
('Omaha', 69.07142857142857)
('High Point', 69.07692307692308)
('Delaware St.', 69.13333333333334)
('Nicholls', 69.14285714285714)
('North Ala.', 69.14285714285714)
('FGCU', 69.2)
('UNC Asheville', 69.21428571428571)
('Southern U.', 69.21428571428571)
('Air Force', 69.21428571428571)
('Western Ill.', 69.25)
('Army West Point', 69.25)
('California Baptist', 69.27272727272727)
('UTRGV', 69.28571428571429)
('Merrimack', 69.35714285714286)
('Tennessee Tech', 69.36363636363636)
('Southeast Mo. St.', 69.36363636363636)
('Southern Ill.', 69.38461538461539)
('New Orleans', 69.38461538461539)
('Tex. A&M-Commerce', 69.38461538461539)
('Lamar University', 69.38461538461539)
('Milwaukee', 69